In [1]:
import torch
import pickle
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
from copy import deepcopy
from tqdm import tqdm


from model import LR
from data import FairnessDataset, SyntheticDataset, GermanDataset
from ei_effort import Optimal_Effort, PGD_Effort
from ei_utils import *
from ei_model_test import EIModel, fair_batch_proxy, covariance_proxy

# warnings.filterwarnings('ignore')

In [2]:
# dataset = SyntheticDataset(seed=0)
dataset = GermanDataset(seed=0)

In [8]:
torch.manual_seed(0)
train_tensors, val_tensors, test_tensors = dataset.tensor(z_blind=False)
train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)

model = LR(train_dataset.X.shape[1])

tau = 0.5
delta = 0.5
lamb = 0.
# effort = Optimal_Effort(delta)
effort = PGD_Effort(delta, 50)
proxy = covariance_proxy
ei_model = EIModel(model, proxy, effort, tau)
ei_model.train(train_dataset, lamb, alpha=0, abstol=1e-3)

for module in model.layers:
    if hasattr(module, 'weight'):
        weights_0 = module.weight.data
    if hasattr(module, 'bias'):
        bias_0 = module.bias.data

theta_0 = torch.cat((weights_0[0], bias_0), 0)
theta_0

 Training [alpha=0.00; lambda=0.00; delta=0.50]: 100%|██████████| 100/100 [00:03<00:00, 29.61epochs/s]


tensor([ 0.4047, -0.3873,  0.1086, -0.0497, -0.0455,  0.2492,  0.0892, -0.0266,
         0.0283,  0.2969, -0.0398, -0.2192, -0.0503, -0.0040, -0.0189,  0.0487,
         0.0349,  0.3141,  0.3925, -0.0152,  0.0889])

In [9]:
theta_0_r - theta_0

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
for alpha in [0., 0.1, 0.5, 1.5, 2., 2.5, 5.0]:
    pga_Y_hat, pga_Y_hat_max, pga_fair_loss = ei_model.predict(test_dataset, alpha, 1e-7)
    pga_accuracy, pga_ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), pga_Y_hat, pga_Y_hat_max, tau)
    for module in ei_model.model_adv.layers:
        if hasattr(module, 'weight'):
            pga_weights_r = module.weight.data
        if hasattr(module, 'bias'):
            pga_bias_r = module.bias.data
    pga_theta_r = torch.cat((pga_weights_r[0], pga_bias_r), 0)

    Y_neg = pga_Y_hat<tau
    
    alphas = (pga_theta_r-theta_0).abs()
    print(f'[{dataset.__class__.__name__}] PGA Results {"(Covariance)" if proxy == covariance_proxy else "(Loss-based)"}')
    print(f'alpha               |   {alpha}')
    print(f'Accuracy            |   {pga_accuracy:.5f}')
    print(f'Fairness Loss       |   {pga_fair_loss:.5f}')
    print(f'EI Disparity        |   {pga_ei_disparity:.5f}')
    # print(f'theta_0             |   {np.round(theta_0, 4)}')
    # print(f'theta_r             |   {np.round(pga_theta_r, 4)}')
    print(f'alphas              |   {np.round(alphas, 4)}')
    print('-'*32)
    print()

In [ ]:
for alpha in [0., 0.1, 0.5, 1.5, 2., 2.5, 5.0]:
    pga_Y_hat, pga_Y_hat_max, pga_fair_loss = ei_model.predict(test_dataset, alpha, 1e-7)
    pga_accuracy, pga_ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), pga_Y_hat, pga_Y_hat_max, tau)
    for module in ei_model.model_adv.layers:
        if hasattr(module, 'weight'):
            pga_weights_r = module.weight.data
        if hasattr(module, 'bias'):
            pga_bias_r = module.bias.data
    pga_theta_r = torch.cat((pga_weights_r[0], pga_bias_r), 0)

    Y_neg = pga_Y_hat<tau
    
    alphas = (pga_theta_r-theta_0).abs()
    print(f'[{dataset.__class__.__name__}] PGA Results {"(Covariance)" if proxy == covariance_proxy else "(Loss-based)"}')
    print(f'alpha               |   {alpha}')
    print(f'Accuracy            |   {pga_accuracy:.5f}')
    print(f'Fairness Loss       |   {pga_fair_loss:.5f}')
    print(f'EI Disparity        |   {pga_ei_disparity:.5f}')
    # print(f'theta_0             |   {np.round(theta_0, 4)}')
    # print(f'theta_r             |   {np.round(pga_theta_r, 4)}')
    print(f'alphas              |   {np.round(alphas, 4)}')
    print('-'*32)
    print()